In [8]:
import numpy as np
import imutils
import cv2

class Stitcher:
    def __init__(self):
        self.isv3 = imutils.is_cv3(or_better=True)
    def stitch(self, images, ratio=0.75, Threshold=4.0, showMatches=False):
        (imageA, imageB) = images
        (kpsA, featuresA) = self.detectAndDescribe(imageA)
        (kpsB, featuresB) = self.detectAndDescribe(imageB)
        M = self.matchKeypoints(kpsA, kpsB,featuresA, featuresB, imageA, imageB)
        (matches, H, status, vis) = M
        result = cv2.warpPerspective(imageB, H, (imageB.shape[1] + imageA.shape[1], imageA.shape[0]))
        result[0:imageA.shape[0], 0:imageA.shape[1]] = imageA
        return (result,vis,H)

    def detectAndDescribe(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        descriptor = cv2.ORB_create()
        (kps, features) = descriptor.detectAndCompute(gray, None)
        return (kps, features)

    def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
                ratio, Threshold):
        matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
        matches = matcher.match(featuresA, featuresB, None)
        matches.sort(key=lambda x: x.distance, reverse=False)
        GoodMatches = int(len(matches) * 0.25)
        matches = matches[:GoodMatches]
        print("The best distance between the descriptors is :")
        print(cv2.DMatch().distance)
        ptsA = np.zeros((len(matches), 2), dtype=np.float32)
        ptsB = np.zeros((len(matches), 2), dtype=np.float32)
        v = cv2.drawMatches(imageA, kpsA, imageB, kpsB, matches, None)
        for i, match in enumerate(matches):
            ptsA[i, :] = kpsA[match.queryIdx].pt
            ptsB[i, :] = kpsB[match.trainIdx].pt
        (H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC,4.0)
        return (matches, H, status, v)

    def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        v = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        v[0:hA, 0:wA] = imageA
        v[0:hB, wA:] = imageB
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            if s == 1:
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(v, ptA, ptB, (0, 255, 0), 1)
        return v
    
def trim(frame):
    result = cv2.copyMakeBorder(frame, 50, 50, 50, 50, cv2.BORDER_CONSTANT, (0, 0, 0))
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    (x, y, w, h) = cv2.boundingRect(c)
    result = result[y:y + h, x:x + w]   
    return result

imageA = cv2.imread('BigFour0.png',1)
imageB = cv2.imread('BigFour1.png',1)
imageC = cv2.imread('BigFour2.png',1)
stitcher = Stitcher()

(result, v, h) = stitcher.stitch([imageA, imageB], showMatches=True)
result = trim(result)
(result1, v1, h1) = stitcher.stitch([result, imageC], showMatches=True)
result1 = trim(result1)
cv2.imwrite("Output.png",result1)
cv2.imshow("Output",result1)
cv2.waitKey(0)

The best distance between the descriptors is :
3.4028234663852886e+38
The best distance between the descriptors is :
3.4028234663852886e+38


-1